<a href="https://colab.research.google.com/github/ewu2023/CS589-HW4/blob/main/CS589_NeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello")